# Friendship at school
In this notebook we analize the networks of students belonging to different types of schools, calculating various metrices and tools. Additional information about everything is provided in the projext report.<br>
By:
- Guglielmo Biagini
- Elisa Castagnari
- Matteo Fusconi
- Luca Trambaiollo


<a target="_blank" href="https://colab.research.google.com/github/MatteoFusconi/Social-Network-Analysis/blob/main/FriendshipAtSchool.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

 #### setup

In [28]:
try:
  import google.colab
  !git clone https://github.com/MatteoFusconi/Social-Network-Analysis.git
  data_folder = './Social-Network-Analysis/data/'
except ImportError:
  data_folder = './data/'
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os

## Primary School

In [30]:
input_file = data_folder + 'primary_data.txt'

## Middle School

## High School

In [31]:
input_file = data_folder + 'high_data.csv'
contacts = pd.read_csv(input_file, sep= " ", names= ["IDcontact", "ID1", "ID2", "Class1", "Class2"])
np.sort(pd.unique(contacts["ID1"]))

array([   1,    3,    4,    9,   14,   15,   20,   21,   27,   28,   32,
         34,   38,   39,   41,   45,   46,   47,   48,   54,   55,   58,
         61,   62,   63,   65,   70,   71,   72,   79,   80,   85,   87,
         89,   90,   92,  101,  103,  106,  111,  115,  116,  117,  119,
        120,  122,  124,  125,  131,  132,  134,  145,  147,  149,  151,
        156,  159,  160,  165,  170,  171,  172,  173,  176,  177,  179,
        181,  184,  185,  190,  194,  196,  199,  200,  201,  202,  205,
        209,  210,  211,  213,  214,  219,  220,  222,  226,  227,  232,
        236,  239,  240,  241,  242,  243,  245,  246,  248,  252,  253,
        254,  255,  256,  257,  258,  265,  267,  268,  269,  272,  274,
        275,  276,  277,  279,  280,  285,  290,  295,  306,  311,  312,
        322,  325,  327,  333,  335,  339,  343,  346,  353,  356,  364,
        366,  372,  374,  376,  382,  387,  388,  390,  391,  400,  407,
        413,  420,  424,  425,  428,  429,  434,  4

## University